# Machine Learning Crash Course
## Bornhack 2019

In [1]:
!pip freeze

appnope==0.1.0
astroid==2.2.5
attrs==19.1.0
backcall==0.1.0
bleach==3.1.0
decorator==4.4.0
defusedxml==0.6.0
entrypoints==0.3
ipykernel==5.1.1
ipython==7.6.1
ipython-genutils==0.2.0
isort==4.3.21
jedi==0.14.0
Jinja2==2.10.1
json5==0.8.4
jsonschema==3.0.1
jupyter-client==5.2.4
jupyter-core==4.5.0
jupyterlab==1.0.1
jupyterlab-server==1.0.0
lazy-object-proxy==1.4.1
MarkupSafe==1.1.1
mccabe==0.6.1
mistune==0.8.4
nbconvert==5.5.0
nbformat==4.4.0
notebook==5.7.8
pandocfilters==1.4.2
parso==0.5.0
pexpect==4.7.0
pickleshare==0.7.5
prometheus-client==0.7.1
prompt-toolkit==2.0.9
ptyprocess==0.6.0
Pygments==2.4.2
pylint==2.3.1
pyrsistent==0.15.2
python-dateutil==2.8.0
pyzmq==18.0.2
Send2Trash==1.5.0
six==1.12.0
terminado==0.8.2
testpath==0.4.2
tornado==6.0.3
traitlets==4.3.2
typed-ast==1.4.0
wcwidth==0.1.7
webencodings==0.5.1
wrapt==1.11.2
